# kaggle Data Sets

Combining public country statistics from th US Census Bureau with the Human Freedom Index
* United States International Census (Google BigQuery): https://www.kaggle.com/census/census-bureau-international/metadata
* Human Freedom Index (csv): https://www.kaggle.com/gsutters/the-human-freedom-index

ETL Repository https://github.com/Jmaddalena42/ETL_Project

## Obtain Census Dataset

BigQuery Public Datasets https://cloud.google.com/bigquery/public-data/

### Enable Google Cloud services

Google Cloud Python Service Quick Start https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries#client-libraries-install-python

### Install Dependencies

$ pip install google.cloud

$ pip install --upgrade google-cloud-bigquery

$ pip install --upgrade google-cloud-storage

### Get Service Account API Credentials

Download Service Account Key

https://console.cloud.google.com/apis/credentials?project=bootcamp-1569375678077&folder&organizationId

Enable BigQuery API

https://console.developers.google.com/apis/api/bigquery-json.googleapis.com/overview?project=bootcamp-1569375678077

### Enable Google Credentials for every session

GOOGLE_APPLICATION_CREDENTIALS must be set before jupyter notebook is started every session:

$ set GOOGLE_APPLICATION_CREDENTIALS='C:\Users\janin\OneDrive\Documents\GitHub\ETL_Project\Bootcamp-f0347722c83b.json'



In [ ]:
# Constants
PROJECT_ID = 'bigquery-public-data' # All BigQuery public datasets use this project ID
dataset = 'census_bureau_international' # Name of the dataset we're interested in

# Dependencies to be able to get data from Google BigQuery
from google.cloud import bigquery
bigquery_client = bigquery.Client(project=PROJECT_ID)
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [7]:
# Get dataset reference
census_dataset_ref = bigquery_client.dataset(dataset, project=PROJECT_ID)

In [9]:
# Get dataset
census_dset = bigquery_client.get_dataset(census_dataset_ref)

To resolve  403 error, enabled BigQuery API at https://console.developers.google.com/apis/library/bigquery-json.googleapis.com?project=bootcamp-1569375678077&pli=1.

Example error:

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/datasets/census_bureau_international: BigQuery API has not been used in project 488912422205 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/bigquery-json.googleapis.com/overview?project=488912422205 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.

In [10]:
# List tables in the dataset
[x.table_id for x in bigquery_client.list_tables(census_dset)]

['age_specific_fertility_rates',
 'birth_death_growth_rates',
 'country_names_area',
 'midyear_population',
 'midyear_population_5yr_age_sex',
 'midyear_population_age_sex',
 'midyear_population_agespecific',
 'mortality_life_expectancy']